Objectives
==========

In , you were introduced to the concept of discretization, and saw that
there were many different ways to approximate a given problem. This Lab
will delve further into the concepts of accuracy and stability of
numerical schemes, in order that we can compare the many possible
discretizations.

At the end of this Lab, you will have seen where the error in a
numerical scheme comes from, and how to quantify the error in terms of
*order*. The stability of several examples will be demonstrated, so that
you can recognize when a scheme is unstable, and how one might go about
modifying the scheme to eliminate the instability.

Specifically you will be able to:

-   Define the term and identify: Implicit numerical scheme and Explicit
    numerical scheme.

-   Define the term, identify, or write down for a given equation:
    Backward Euler method and Forward Euler method.

-   Explain the difference in terminology between: Forward difference
    discretization and Forward Euler method.

-   Define: truncation error, local truncation error, global truncation
    error, and stiff equation.

-   Explain: a predictor-corrector method.

-   Identify from a plot: an unstable numerical solution.

-   Be able to: find the order of a scheme, using the test equation find
    the stability of a scheme, find the local truncation error from a
    graph of the exact solution and the numerical solution.

Readings
========

This lab is designed to be entirely self-contained. If you would like
additional background on any of the following topics, then refer to the
indicated sections in the references.

-   **Differential Equations:**

    -    @strang-am, Chapter 6 (ODE’s).

-   **Numerical Methods:**

    -    @strang-am, Section 6.5 (a great overview of difference methods
        for initial value problems)

    -    @burden-faires, Chapter 5 (a more in-depth analysis of the
        numerical methods and their accuracy and stability).

Introduction {#lab2:sec:intro}
============

Remember from Lab \#1  that you were introduced to three approximations
to the first derivative of a function, $T^\prime(t)$. If the independent
variable, $t$, is discretized at a sequence of N points,
$t_i=t_0+i \Delta t$, where $i
= 0,1,\ldots, N$ and $\Delta t= 1/N$, then we can write the three
approximations as follows:

 **Forward difference formula:**

:   $$T^\prime(t_i) \approx \frac{T_{i+1}-T_i}{\Delta t},
        \label{lab2:eq:forward-diff}$$

 **Backward difference formula:**

:   $$T^\prime(t_i) \approx \frac{T_{i}-T_{i-1}}{\Delta t},
        \label{lab2:eq:backward-diff}$$

 **Centered difference formula:**

:   $$T^\prime(t_i) \approx \frac{T_{i+1}-T_{i-1}}{2 \Delta t}.
        \label{lab2:eq:centered-diff}$$

In fact, there are many other possible methods to approximate the
derivative (some of which we will see later in this Lab). With this
large choice we have in the choice of approximation scheme, it is not at
all clear at this point which, if any, of the schemes is the “best”. It
is the purpose of this Lab to present you with some basic tools that
will help you to decide on an appropriate discretization for a given
problem. There is no generic “best” method, and the choice of
discretization will always depend on the problem that is being dealt
with.

In an example from Lab \#1, the forward difference formula was used to
compute solutions to the saturation development equation, and you saw
two important results:

-   reducing the grid spacing, $\Delta t$, seemed to improve the
    accuracy of the approximate solution; and

-   if $\Delta t$ was taken too large (that is, the grid was not fine
    enough), then the approximate solution exhibited non-physical
    oscillations, or a *numerical instability*.

There are several questions that arise from this example:

1.  Is it always true that reducing $\Delta t$ will improve the discrete
    solution?

2.  Is it possible to improve the accuracy by using another
    approximation scheme (such as one based on the backward or centered
    difference formulas)?

3.  Are these numerical instabilities something that always appear when
    the grid spacing is too large?

4.  By using another difference formula for the first derivative, is it
    possible to improve the stability of the approximate solution, or to
    eliminate the stability altogether?

The first two questions, related to *accuracy*, will be dealt with in
Section [lab2:sec:accuracy-main], and the last two will have to wait
until Section [lab2:sec:stability] when *stability* is discussed.
    
Accuracy of Difference Approximations {#lab2:sec:first-deriv}
=====================================

[lab2:sec:accuracy-main]

Before moving on to the details of how to measure the error in a scheme,
let’s take a closer look at another example which we’ve seen already …

[lab2:exm:accuracy] Let’s go back to the heat conduction equation from
Lab \#1, where the temperature, $T(t)$, of a rock immersed in water or
air, evolves in time according to the first order ODE:
$$\frac{dT}{dt} = \lambda(T,t) \, (T-T_a) 
    \label{lab2:eq:conduction}$$ with initial condition $T(0)$. We saw
in the section on the forward Euler method  that one way to discretize
this equation was using the forward difference formula  for the
derivative, leading to
$$T_{i+1} = T_i + \Delta t \, \lambda(T_i,t_i) \, (T_i-T_a).$$
Similarly, we could apply either of the other two difference formulae,
or , to obtain other difference schemes, namely what we called the
*backward Euler method*
$$T_{i+1} = T_i + \Delta t \, \lambda(T_{i+1},t_{i+1}) \, (T_{i+1}-T_a),$$
and the *leap frog method*
$$T_{i+1} = T_{i-1} + 2 \Delta t \, \lambda(T_{i},t_{i}) \, (T_{i}-T_a).$$
The forward Euler and leap frog schemes are called *explicit methods*,
since they allow the temperature at any new time to be computed in terms
of the solution values at previous time steps. The backward Euler
scheme, on the other hand, is called an *implicit scheme*, since it
gives an equation defining $T_{i+1}$ implicitly (If $\lambda$ depends
non-linearly on $T$, then this equation may require an additional step,
involving the iterative solution of a non-linear equation. We will pass
over this case for now, and refer you to a reference such as
 @burden-faires for the details on non-linear solvers such as *Newton’s
method*).

For now, let’s assume that $\lambda$ is a constant, independent of $T$
and $t$. Plots of the numerical results from each of these schemes,
along with the exact solution, are given in Figure [lab2:fig:compare]
(with the “unphysical” parameter value $\lambda=0.8$ chosen to enhance
the show the growth of numerical errors, even though in a real material
this would violate conservation of energy).

![A plot of the exact and computed solutions for the temperature of a
rock, with parameters: $T_a=20$, $T(0)=30$, $\lambda= +0.8$,
$\Delta t=\frac{1}{3}$.<span
data-label="lab2:fig:compare"></span>](accuracy/compare1)

Notice from these results that the centered scheme is the most accurate,
and backward Euler the least accurate.

The next section explains why some schemes are more accurate than
others, and introduces a means to quantify the accuracy of a numerical
approximation.

Round-off Error and Discretization Error {#lab2:sec:accuracy}
----------------------------------------

From Example [lab2:exm:accuracy] and the example in the Forward Euler
section of the previous lab,  it is obvious that a numerical
approximation is exactly that - **an approximation**. The process of
discretizing a differential equation inevitably leads to errors. In this
section, we will tackle two fundamental questions related to the
accuracy of a numerical approximation:

-   Where does the error come from (and how can we measure it)?

-   How can the error be controlled?

### Where does the error come from?

#### Round-off error:

When attempting to solve differential equations on a computer, there are
two main sources of error. The first, *round-off error*, derives from
the fact that a computer can only represent real numbers by *floating
point* approximations, which have only a finite number of digits of
accuracy.

For example, we all know that the number $\pi$ is a non-repeating
decimal, which to the first twenty significant digits is
$3.1415926535897932385\dots$ Imagine a computer which stores only eight
significant digits, so that the value of $\pi$ is rounded to
$3.1415927$.

In many situations, these five digits of accuracy may be sufficient.
However, in some cases, the results can be catastrophic, as shown in the
following example: $$\frac{\pi}{(\pi + 0.00000001)-\pi}.$$ Since the
computer can only “see” 8 significant digits, the addition
$\pi+0.00000001$ is simply equal to $\pi$ as far as the computer is
concerned. Hence, the computed result is $\frac{1}{0}$ - an undefined
expression! The exact answer $100000000\pi$, however, is a very
well-defined non-zero value.

#### Truncation error:

The second source of error stems from the discretization of the problem,
and hence is called *discretization error* or *truncation error*. In
comparison, round-off error is always present, and is independent of the
discretization being used. The simplest and most common way to analyse
the truncation error in a scheme is using *Taylor series expansions*.

Let us begin with the forward difference formula for the first
derivative, , which involves the discrete solution at times $t_{i+1}$
and $t_{i}$. Since only continuous functions can be written as Taylor
series, we expand the exact solution (instead of the discrete values
$T_i$) at the discrete point $t_{i+1}$:
$$T(t_{i+1}) = T(t_i+\dt) = T(t_i) + (\Delta t) T^\prime(t_i) + 
  \frac{1}{2}(\Delta t)^2 T^{\prime\prime}(t_i) +\cdots,
%%+   \frac{1}{6}(\dt)^3   y^{\prime\prime\prime}(t_i) + \cdots$$ We can
then subtract $T(t_i)$ and divide the result by $\Delta t$ in order to
obtain $$\begin{aligned}
  \frac{T(t_{i+1})-T(t_i)}{\Delta t} &=& T^{\prime}(t_i) +
  \underbrace{\frac{1}{2}(\Delta t) T^{\prime\prime}(t_i) + \cdots}
_{\mbox{\rm truncation error}} \nonumber \\ \; \nonumber \\
%%+    \frac{1}{6}(\dt)^2   y^{\prime\prime\prime}(t_i) + \cdots} 
  &=& T^{\prime}(t_i) + {\cal O}(\Delta t).
  \label{lab2:eq:disc-error}\end{aligned}$$

This second expression writes the truncation error term in terms of
*order notation*. If we write $y = {\cal O}(\Delta t)$, then we mean
simply that $y<c \cdot \Delta t$ for some constant $c$, and we say that
“ *$y$ is first order in $\Delta t$*” (since it depends on $\Delta t$ to
the first power) or “ *$y$ is big-oh of $\Delta t$*.” As $\Delta t$ is
assumed small, the next term in the series, $\Delta t^2$ is small
compared to the $\Delta t$ term.

It is clear from that as $\Delta t$ is reduced in size (as the
computational grid is refined), the error is also reduced. If you
remember that we derived the approximation from the limit definition of
derivative, then this should make sense. This dependence of the error on
powers of the grid spacing $\Delta t$ is an underlying characteristic of
difference approximations, and we will see approximations with higher
orders in the coming sections …

There is one more important distinction to be made here. The “truncation
error” we have been discussing so far is actually what is called *local
truncation error*. It is “local” in the sense that we have expanded the
Taylor series *locally* about the exact solution at the point $t_i$.

There is also a *global truncation error* (or, simply, *global error*),
which is the error made during the course of the entire computation,
from time $t_0$ to time $t_n$. The difference between local and global
truncation error is illustrated in Figure [lab2:fig:truncation-error].

![Local and global truncation error. <span
data-label="lab2:fig:truncation-error"></span>](images/error)

It is easy to get a handle on the order of the local truncation error
using Taylor series, regardless of whether the exact solution is known,
but no similar analysis is available for the global error. We can write
$$\mbox{\rm global error} = |T(t_n)-T_n|$$ (see
Figure [lab2:fig:truncation-error]), but this expression can only be
evaluated if the exact solution is known ahead of time (which is not the
case in most problems we want to compute, since otherwise we wouldn’t be
computing it in the first place!). Therefore, when we refer to
truncation error, we will always be referring to the local truncation
error.

[lab2:prob:taylor]

-   a\) Derive the error term for the backward difference formula using
    Taylor series, and hence show that it is also first order.

-   b\) How does the constant in front of the leading order error term differ
    from that for the forward difference formula? Relate this back to the
    results plotted in Figure [lab2:fig:compare], where these two formulae
    were used to derive difference schemes for the heat conduction problem.

-   c\) Repeat a) for the centered difference formula . What is the order of
    this approximation to the first derivative?

### How can we control the error?

Now that we’ve determined the source of the error in numerical methods,
we would like to find a way to control it; that is, we would like to be
able to compute and be confident that our approximate solution is
“close” to the exact solution. Round-off error is intrinsic to all
numerical computations, and cannot be controlled (except to develop
methods that do not magnify the error unduly … more on this later).
Truncation error, on the other hand, *is* under our control.

In the simple ODE examples that we’re dealing with in this lab, the
round-off error in a calculation is much smaller than the truncation
error. Furthermore, the schemes being used are *stable with respect to
round-off error* in the sense that round-off errors are not magnified in
the course of a computation. So, we will restrict our discussion of
error control in what follows to the truncation error.

However, there are many numerical algorithms in which the round-off
error can dominate the the result of a computation (Gaussian elimination
is one example, which you will see in Lab \#3 ), and so we must always
keep it in mind when doing numerical computations.

There are two fundamental ways in which the truncation error in an
approximation  can be reduced:

1.  Decrease the grid spacing, . Provided that the second derivative of
    the solution is bounded, it is clear from the error term in  that as
      is reduced, the error will also get smaller. This principle was
    demonstrated in an example from Lab \#1. The disadvantage to
    decreasing  is that the cost of the computation increases since more
    steps must be taken. Also, there is a limit to how small  can be,
    beyond which round-off errors will start polluting the computation.

2.  Increase the order of the approximation. We saw above that the
    forward difference approximation of the first derivative is first
    order accurate in the grid spacing. It is also possible to derive
    higher order difference formulas which have a leading error term of
    the form $(\dt)^p$, with $p>1$. You should have discovered already
    in Problem [lab2:prob:taylor] that the centered difference formula
    is a second order scheme, and some further examples will be given in
    Section [lab2:sec:highorder-taylor]. The main disadvantage to using
    very high order schemes is that the error term depends on higher
    derivatives of the solution, which can sometimes be very large – in
    this case, the stability of the scheme can be adversely affected
    (for more on this, see Section [lab2:sec:stability]).

[lab2:prob:accuracy] In order to investigate these two approaches to
improving the accuracy of an approximation, you can use the code in
terror2.m or terror2.py to play with the solutions to the heat
conduction equation. For a given function $\lambda(T)$, and specified
parameter values, you should experiment with various time steps and
schemes, and compare the computed results (Note: only the answers to the
assigned questions need to be handed in). Look at the different schemes
(euler, leapfrog, 4th order runge kutta) run them for various total
times (tend) and step sizes (dt=tend/npts).

The three schemes that will be used here are forward Euler (first
order), leap frog (second order) and the fourth order Runge-Kutta scheme
(which will be introduced in ).

To hand in: Try three different step sizes for all three schemes for a
total of 9 runs. It’s helpful to be able to change the axis limits to
look at various parts of the plot. In matlab you do that with a sequence
like

    figure(1);  %make figure 1 current
    ax=gca;     %get the current  axis
    set(ax,'ylim',[-10,50]);  %set the y limits
    set(ax,'xlim',[0,1]);  % zoom in on the first second

Use your 9 results to answer parts a and b below. (The leap-frog scheme
should give you quasi-pathological results, see the explanation at the
end of section 5)

-   a\) Does increasing the order of the scheme, or decreasing the time step
    always improve the solution?

-   b\) How would you compute the local truncation error from the error plot?
    And the global error? Do this on a plot for one set of parameters.

-   c\) Similarly, how might you estimate the *order* of the local truncation
    error? The order of the global error? ( **Hint:** An order $p$ scheme
    has truncation error that looks like $c\cdot(\Delta t)^p$. Read the
    error off the plots for several values of the grid spacing and use this
    to find $p$.) Are the local and global error significantly different?
    Why or why not?

Other Approximations to the First Derivative {#lab2:sec:other}
--------------------------------------------

The Taylor series method of deriving difference formulae for the first
derivative is the simplest, and can be used to obtain approximations
with even higher order than two. There are also many other ways to
discretize the derivatives appearing in ODE’s, as shown in the following
sections…

### Higher Order Taylor Methods {#lab2:sec:highorder-taylor}

As mentioned earlier, there are many other possible approximations to
the first derivative using the Taylor series approach. The basic
approach in these methods is as follows:

1.  expand the solution in a Taylor series at one or more points
    surrounding the point where the derivative is to be approximated
    (for example, for the centered scheme, you used two points,
    $T(t_i+\dt)$ and $T(t_i-\dt)$. You also have to make sure that you
    expand the series to a high enough order …

2.  take combinations of the equations until the $T_i$ (and possibly
    some other derivative) terms are eliminated, and all you’re left
    with is the first derivative term.

One example is the fourth-order centered difference formula for the
first derivative:
$$\frac{-T(t_{i+2})+8T(t_{i+1})-8T(t_{i-1})+T(t_{i-2})}{12\dt} =
  T^\prime(t_i) + {\cal O}((\dt)^4)$$

**Quiz:** Try the quiz at [this
link](http://clouds.eos.ubc.ca/~phil/numeric/docs/_build/html/quizzes2/order.html)
related to this higher order scheme.

### Predictor-Corrector Methods {#lab2:sec:pred-corr}

Another class of discretizations are called *predictor-corrector
methods*. Implicit methods can be difficult or expensive to use because
of the solution step, and so they are seldom used to integrate ODE’s.
Rather, they are often used as the basis for predictor-corrector
algorithms, in which a “prediction” for $T_{i+1}$ based only on an
explicit method is then “corrected” to give a better value by using this
precision in an implicit method.

To see the basic idea behind these methods, let’s go back (once again)
to the backward Euler method for the heat conduction problem which
reads:
$$T_{i+1} = T_{i} + \Delta t \, \lambda( T_{i+1}, t_{i+1} ) \, ( T_{i+1}
- T_a ).$$ Note that after applying the backward difference formula ,
all terms in the right hand side are evaluated at time $t_{i+1}$.

Now, $T_{i+1}$ is defined implicitly in terms of itself, and unless
$\lambda$ is a very simple function, it may be very difficult to solve
this equation for the value of $T$ at each time step. One alternative,
mentioned already, is the use of a non-linear equation solver such as
Newton’s method to solve this equation. However, this is an iterative
scheme, and can lead to a lot of extra expense. A cheaper alternative is
to realize that we could try estimating or *predicting* the value of
$T_{i+1}$ using the simple explicit forward Euler formula and then use
this in the right hand side, to obtain a *corrected* value of $T_{i+1}$.
The resulting scheme, $$\begin{array}{ll}
  \mbox{\rm\bf Prediction:} & \widetilde{T}_{i+1} = T_i + \dt \,
  \lambda(T_i,t_i) \, (T_i-T_a), \\ \; \\
  \mbox{\rm\bf Correction:} & T_{i+1} = T_i + \dt \,
  \lambda(\widetilde{T}_{i+1},t_{i+1}) \, (\widetilde{T}_{i+1}-T_a).
\end{array}$$

This method is an explicit scheme, which can also be shown to be second
order accurate in . It is the simplest in a whole class of schemes
called *predictor-corrector schemes* (more information is available on
these methods in a numerical analysis book such as  @burden-faires).

### Other Methods {#lab2:sec:other-methods}

The choice of methods is made even greater by two other classes of
schemes:

 **Runge-Kutta methods:**

:   which will be described in Lab \#4.

 **Multi-step methods:**

:   which use values of the solution at more than one previous time step
    in order to increase the accuracy. Compare these to one-step
    schemes, such as forward Euler, which use the solution only at one
    previous step.

More can be found on these (and other) methods in  @burden-faires.

### Summary

In this section, you’ve been given a short overview of the accuracy of
difference schemes for first order ordinary differential equations.
We’ve seen that accuracy can be improved by either decreasing the grid
spacing, or by choosing a higher order scheme from one of several
classes of methods. When using a higher order scheme, it is important to
realize that the cost of the computation usually rises due to an added
number of function evaluations (especially for multi-step and
Runge-Kutta methods). When selecting a numerical scheme, it is important
to keep in mind this trade-off between accuracy and cost.

However, there is another important aspect of discretization that we
have pretty much ignored. The next section will take a look at schemes
of various orders from a different light, namely that of *stability*.

Stability of Difference Approximations {#lab2:sec:stability}
======================================

The easiest way to introduce the concept of stability is for you to see
it yourself.

[lab2:prob:stability] This example is a slight modification of
Problem [lab2:prob:accuracy] from the previous section on accuracy. We
will add one scheme (backward euler) and drop the 4th order Runge-Kutta,
and change the focus from error to stability. The value of $\lambda$ is
assumed a constant, so that the backward Euler scheme results in an
explicit method, and we’ll also compute a bit further in time, so that
any instability manifests itself more clearly. Run the stability2.m or
stability2.py scripts with $\lambda= -8\ s^{-1}$, with $\Delta t$ values
that just straddle the stability condition for the forward euler scheme
($\Delta t < \frac{-2}{\lambda}$, derived below). Hand in plots with
comments that show that 1) the stability condition does in fact predict
the onset of the instablity in the euler scheme, and 2) the backward
euler and leapfrog are either stable or unstable for the same $\Delta t$
values. (you should run out to longer than tend=10 seconds to see if
there is a delayed instability.)

The heat conduction problem, as you saw in Lab \#1, has solutions that
are stable when $\lambda<0$. It is clear from
Problem [lab2:prob:stability] above that some higher order schemes
(namely, the leap-frog scheme) introduce a spurious oscillation not
present in the continuous solution. This is called a *computational* or
*numerical instability*, because it is an artifact of the discretization
process only. This instability is not a characteristic of the heat
conduction problem alone, but is present in other problems where such
schemes are used. Furthermore, as we will see below, even a scheme such
as forward Euler can be unstable for certain problems and choices of the
time step.

There is a way to determine the stability properties of a scheme, and
that is to apply the scheme to the *test equation*
$$\frac{dz}{dt} = \lambda z, \label{lab2:eq:test-equation}$$ where
$\lambda$ is a complex constant.

The reason for using this equation may not seem very clear. But if you
think in terms of $\lambda z$ as being the linearization of some more
complex right hand side, then the solution to is $z=e^{\lambda t}$, and
so $z$ represents, in some sense, a Fourier mode of the solution to the
linearized ODE problem. We expect that the behaviour of the simpler,
linearized problem should mimic that of the original problem.

Applying the forward Euler scheme to this test equation, results in the
following difference formula $$z_{i+1} = z_i+(\lambda \Delta t)z_i$$
which is a formula that we can apply iteratively to $z_i$ to obtain
$$\begin{aligned}
z_{i+1} &=& (1+\lambda \Delta t)z_{i} \\
        &=& (1+\lambda \Delta t)^2 z_{i-1} \\
        &=& \cdots \\
        &=& (1+\lambda \Delta t)^{i+1} z_{0}.\end{aligned}$$ The value
of $z_0$ is fixed by the initial conditions, and so this difference
equation for $z_{i+1}$ will “blow up” as $i$ gets bigger, if the factor
in front of $z_0$ is greater than 1 in magnitude – this is a sign of
instability. Hence, this analysis has led us to the conclusion that if
$$|1+\lambda\Delta t| < 1,$$ then the forward Euler method is stable.
For *real* values of $\lambda<0$, this inequality can be shown to be
equivalent to the *stability condition*
$$\Delta t < \frac{-2}{\lambda},$$ which is a restriction on how large
the time step can be so that the numerical solution is stable.

[lab2:prob:test-backward-euler] Perform a similar analysis for the
backward Euler formula, and show that it is *always stable* when
$\lambda$ is real and negative.

[lab2:exm:test-leap-frog] *Now, what about the leap frog scheme?*

Applying the test equation to the leap frog scheme results in the
difference equation $$z_{i+1} = z_{i-1} + 2 \lambda \Delta t z_i.$$
Difference formulas such as this one are typically solved by looking for
a solution of the form $z_i = w^i$ which, when substituted into this
equation, yields $$w^2 - 2\lambda\Delta t w - 1 = 0,$$ a quadratic
equation with solution
$$w = \lambda \Delta t \left[ 1 \pm \sqrt{1+\frac{1}{(\lambda
        \Delta t)^2}} \right].$$ The solution to the original difference
equation, $z_i=w^i$ is stable only if all solutions to this quadratic
satisfy $|w|<1$, since otherwise, $z_i$ will blow up as $i$ gets large.

The mathematical details are not important here – what is important is
that there are two (possibly complex) roots to the quadratic equation
for $w$, and one is *always* greater than 1 in magnitude *unless*
$\lambda$ is pure imaginary ( has real part equal to zero), *and*
$|\lambda \Delta t|<1$. For the heat conduction equation in
Problem [lab2:prob:stability] (which is already of the same form as the
test equation ), $\lambda$ is clearly not imaginary, which explains the
presence of the instability for the leap-frog scheme.

Nevertheless, the leap frog scheme is still useful for computations. In
fact, it is often used in geophysical applications, as you will see
later on when discretizing , and .

An example of where the leap frog scheme is superior to the other first
order schemes is for undamped periodic motion (which arose in the
weather balloon example from Lab \#1 ). This corresponds to the system
of ordinary differential equations (with the damping parameter, $\beta$,
taken to be zero): $$\frac{dy}{dt} = u,$$
$$\frac{du}{dt} = - \frac{\gamma}{m} y.$$ You’ve already discretized
this problem using the forward difference formula, and the same can be
done with the second order centered formula. We can then compare the
forward Euler and leap-frog schemes applied to this problem.

Solution plots are given in Figure [lab2:fig:test-leap-frog], for
parameters $\gamma/m=1$, $\Delta t=0.25$, $y(0)=0.0$ and $u(0)=1.0$, and
demonstrate that the leap-frog scheme is stable, while forward Euler is
unstable. This can easily be explained in terms of the stability
criteria we derived for the two schemes when applied to the test
equation. The undamped oscillator problem is a linear problem with pure
imaginary eigenvalues, so as long as $|\sqrt{\gamma/m}\Delta t|<1$, the
leap frog scheme is stable, which is obviously true for the parameter
values we are given. Furthermore, the forward Euler stability condition
$|1+\lambda\Delta
  t|<1$ is violated for any choice of time step (when $\lambda$ is pure
imaginary) and so this scheme is always unstable for the undamped
oscillator.

![Numerical solution to the undamped harmonic oscillator problem, using
the forward Euler and leap-frog schemes. Parameter values:
$\gamma/m=1.0$, $\Delta t=0.25$, $y(0)=0$, $u(0)=1.0$. The exact
solution is a sinusoidal wave.<span
data-label="lab2:fig:test-leap-frog"></span>](oscillator/leap-frog "fig:")
![Numerical solution to the undamped harmonic oscillator problem, using
the forward Euler and leap-frog schemes. Parameter values:
$\gamma/m=1.0$, $\Delta t=0.25$, $y(0)=0$, $u(0)=1.0$. The exact
solution is a sinusoidal wave.<span
data-label="lab2:fig:test-leap-frog"></span>](oscillator/forward-euler "fig:")

Had we taken a larger time step (such as $\Delta t=2.0$, for example),
then even the leap-frog scheme is unstable. Furthermore, if we add
damping ($\beta\neq 0$), then the eigenvalues are no longer pure
imaginary, and the leap frog scheme is unstable no matter what time step
we use.

Stiff Equations {#lab2:sec:stiffness}
===============

One final note: this Lab has dealt only with ODE’s (and systems of
ODE’s) that are *non-stiff*. *Stiff equations* are equations that have
solutions with at least two widely varying times scales over which the
solution changes. An example of stiff solution behaviour is a problem
with solutions that have rapid, transitory oscillations, which die out
over a short time scale, after which the solution slowly decays to an
equilibrium. A small time step is required in the initial transitory
region in order to capture the rapid oscillations. However, a larger
time step can be taken in the non-oscillatory region where the solution
is smoother. Hence, using a very small time step will result in very
slow and inefficient computations.

There are also many other numerical schemes designed specifically for
stiff equations, most of which are implicit schemes. We will not
describe any of them here – you can find more information in a numerical
analysis text such as  @burden-faires.

Difference Approximations of Higher Derivatives {#lab2:sec:higher-derivs}
===============================================

Higher derivatives can be discretized in a similar way to what we did
for first derivatives. Let’s consider for now only the second
derivative, for which one possible approximation is the second order
centered formula: $$\frac{y(t_{i+1})-2y(t_i)+y(t_{i-1})}{(\dt)^2} = 
  y^{\prime\prime}(t_i) + {\cal O}((\dt)^2),$$ There are, of course,
many other possible formulae that we might use, but this is the most
commonly used.

[lab2:prob:taylor-higher]

-   a\) Use Taylor series to derive this formula.

-   b\) Derive a higher order approximation.

Summary {#lab2:sec:summary}
=======

This lab has discussed the accuracy and stability of difference schemes
for simple first order ODEs. The results of the problems should have
made it clear to you that choosing an accurate and stable discretization
for even a very simple problem is not straightforward. One must take
into account not only the considerations of accuracy and stability, but
also the cost or complexity of the scheme. Selecting a numerical method
for a given problem can be considered as an art in itself.
     

